In [ ]:
!pip install -q torch torchvision torchaudio \
                numpy opencv-python onnxruntime \
                insightface albumentations tqdm \
                matplotlib pandas scikit-learn \
                jsonlines


In [ ]:
import os
import cv2
import torch
import numpy as np
import json
from insightface.app import FaceAnalysis  # ✅ ArcFace 직접 로드

In [ ]:
# ✅ 얼굴 인식 모델 로드 (ArcFace)
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
app.prepare(ctx_id=0 if torch.cuda.is_available() else -1)

In [ ]:
# ✅ 프레임이 저장된 폴더
FRAME_DIR = "/kaggle/input/framestest"
frames = sorted(os.listdir(FRAME_DIR))

In [ ]:
# ✅ 그룹 및 멤버 이름 매핑 (ID → 이름 변환)
GROUPS = ["EXO", "IVE", "NJZ", "RED", "SK", "UNKNOWN"]
MEMBERS = {
    "EXO": ["D.O", "Kai"],
    "IVE": ["An Yujin", "Jang Wonyoung"],
    "NJZ": ["Danielle", "Gang Harin", "Kim Minji", "Pham Hanni"],
    "RED": ["Irene", "Joy", "Seulgi", "Wendy"],
    "SK": ["Hyunjin", "Felix"],
    "UNKNOWN": ["Unknown"]
}

In [ ]:
# ✅ 얼굴 인식 모델 로드 (ArcFace 학습된 모델)
MODEL_PATH = "/kaggle/input/realbest/2best_hierarchical_arcface(c).pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class HierarchicalArcFaceModel(torch.nn.Module):
    def __init__(self, embedding_size=512, num_groups=len(GROUPS), num_members=[len(m) for m in MEMBERS.values()]):
        super(HierarchicalArcFaceModel, self).__init__()
        self.global_fc = torch.nn.Sequential(
            torch.nn.Linear(embedding_size, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
        )
        self.group_head = torch.nn.Linear(512, num_groups)  # 그룹 분류
        self.member_heads = torch.nn.ModuleList(
            [torch.nn.Linear(512, num_members[i]) for i in range(num_groups)]
        )

    def forward(self, x):
        features = self.global_fc(x)
        group_logits = self.group_head(features)  # 그룹 예측
        predicted_group = torch.argmax(group_logits, dim=1)  # 예측된 그룹

        batch_size = features.shape[0]
        member_logits_padded = torch.zeros(batch_size, max([h.out_features for h in self.member_heads])).to(features.device)

        for i in range(batch_size):
            group_idx = predicted_group[i].item()
            member_logits = self.member_heads[group_idx](features[i].unsqueeze(0))
            member_logits_padded[i, :member_logits.shape[1]] = member_logits

        return group_logits, member_logits_padded

In [ ]:
# ✅ 모델 로드
model = HierarchicalArcFaceModel().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

In [ ]:
# ✅ 얼굴 인식 결과 저장 리스트
metadata = []

# ✅ 1️⃣ 얼굴 인식 수행 (프레임별)
for frame_name in frames:
    frame_path = os.path.join(FRAME_DIR, frame_name)
    frame = cv2.imread(frame_path)

    # ✅ ArcFace 모델을 활용하여 얼굴 분석
    faces = app.get(frame)  

    # ✅ 파일명에서 정확히 마지막 숫자(프레임 번호) 추출 (일관성 유지)
    try:
        timestamp = int(frame_name.split("_")[-1].split(".")[0])  # ✅ 마지막 `_숫자.jpg` 부분 추출
    except (IndexError, ValueError):
        print(f"⚠️ 잘못된 파일명 형식: {frame_name} → 기본값 0으로 설정")
        timestamp = 0  # 오류 발생 시 기본값 설정

    if not faces:
        metadata.append({
            "timestamp": timestamp,  # ✅ 동일한 방식 적용
            "group": "UNKNOWN",
            "member": "Unknown",
            "confidence": 0.0,
            "bbox": None
        })
        continue

    for face in faces:
        bbox = face.bbox.astype(int)
        x1, y1, x2, y2 = bbox

        # ✅ 얼굴 임베딩 추출
        embedding = torch.tensor(face.embedding, dtype=torch.float32).unsqueeze(0).to(device)

        # ✅ 모델 예측 (그룹 및 멤버 분류)
        with torch.no_grad():
            group_logits, member_logits = model(embedding)

        # ✅ Softmax 적용 (확률값 변환)
        group_probs = torch.softmax(group_logits, dim=1)
        member_probs = torch.softmax(member_logits, dim=1)

        predicted_group_id = torch.argmax(group_probs, dim=1).item()
        group_confidence = group_probs[0, predicted_group_id].item()

        # ✅ Unknown 판단 (확률 0.7 미만이면 Unknown 처리)
        if group_confidence < 0.7 or predicted_group_id >= len(GROUPS):
            predicted_group_name = "UNKNOWN"
            predicted_member_name = "Unknown"
            member_confidence = group_confidence  # Unknown일 경우 확률 그대로 사용
        else:
            predicted_group_name = GROUPS[predicted_group_id]
            predicted_member_id = torch.argmax(member_probs, dim=1).item()
            member_confidence = member_probs[0, predicted_member_id].item()

            if predicted_group_name in MEMBERS:
                predicted_member_name = MEMBERS[predicted_group_name][predicted_member_id]
            else:
                predicted_member_name = "Unknown"

        # ✅ 하단에서도 동일한 방식으로 timestamp 적용
        metadata.append({
            "timestamp": timestamp,  # ✅ 동일한 timestamp 방식 적용
            "group": predicted_group_name,
            "member": predicted_member_name,
            "confidence": member_confidence,
            "bbox": (x1, y1, x2, y2)
        })

        print(f"📸 {frame_name} 처리 완료 → {predicted_group_name} - {predicted_member_name} ({member_confidence:.2f})")

# ✅ 저장할 디렉토리 설정
save_dir = "/kaggle/working/frame"
os.makedirs(save_dir, exist_ok=True)

# ✅ 변환하여 JSON 저장
def convert_metadata_to_json(metadata, save_path):
    # ✅ 모든 int64 값을 Python의 기본 int 타입으로 변환
    for entry in metadata:
        entry["timestamp"] = int(entry["timestamp"])  # ✅ timestamp 변환
        if "bbox" in entry and entry["bbox"] is not None:
            entry["bbox"] = list(map(int, entry["bbox"]))  # ✅ bbox도 변환
            
# ✅ 파일 저장
save_path = os.path.join(save_dir, "face_metadata.json")
convert_metadata_to_json(metadata, save_path)

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

print(f"✅ 얼굴 인식 결과 저장 완료: {save_path}")


In [ ]:
import os
import json
import cv2
import torch
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm

# ✅ 필수 설정
FRAME_ROOT_DIR = "/kaggle/input/frameid"  # 영상별 프레임이 저장된 루트 폴더
SAVE_METADATA_DIR = "/kaggle/working/metadata"  # 각 영상별 JSON 파일을 저장할 폴더
os.makedirs(SAVE_METADATA_DIR, exist_ok=True)

# ✅ ArcFace 얼굴 인식 모델 로드
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
app.prepare(ctx_id=0 if torch.cuda.is_available() else -1)

# ✅ 학습된 ArcFace 모델 로드
MODEL_PATH = "/kaggle/input/realbest/2best_hierarchical_arcface(c).pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

class HierarchicalArcFaceModel(torch.nn.Module):
    def __init__(self, embedding_size=512, num_groups=6, num_members=[2, 2, 4, 4, 2, 1]):
        super(HierarchicalArcFaceModel, self).__init__()
        self.global_fc = torch.nn.Sequential(
            torch.nn.Linear(embedding_size, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
        )
        self.group_head = torch.nn.Linear(512, num_groups)  # 그룹 분류
        self.member_heads = torch.nn.ModuleList(
            [torch.nn.Linear(512, num_members[i]) for i in range(num_groups)]
        )

    def forward(self, x):
        features = self.global_fc(x)
        group_logits = self.group_head(features)  # 그룹 예측
        predicted_group = torch.argmax(group_logits, dim=1)  # 예측된 그룹

        batch_size = features.shape[0]
        member_logits_padded = torch.zeros(batch_size, max([h.out_features for h in self.member_heads])).to(features.device)

        for i in range(batch_size):
            group_idx = predicted_group[i].item()
            member_logits = self.member_heads[group_idx](features[i].unsqueeze(0))
            member_logits_padded[i, :member_logits.shape[1]] = member_logits

        return group_logits, member_logits_padded

# ✅ 모델 로드
model = HierarchicalArcFaceModel().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ✅ 그룹 및 멤버 정보
GROUPS = ["EXO", "IVE", "NJZ", "RED", "SK", "UNKNOWN"]
MEMBERS = {
    "EXO": ["D.O", "Kai"],
    "IVE": ["An Yujin", "Jang Wonyoung"],
    "NJZ": ["Danielle", "Gang Harin", "Kim Minji", "Pham Hanni"],
    "RED": ["Irene", "Joy", "Seulgi", "Wendy"],
    "SK": ["Hyunjin", "Felix"],
    "UNKNOWN": ["Unknown"]
}

# ✅ 1️⃣ 각 영상별로 프레임을 처리하여 JSON 파일 저장
video_folders = sorted(os.listdir(FRAME_ROOT_DIR))  # ✅ 모든 영상 ID 폴더 가져오기

for video_id in tqdm(video_folders, desc="Processing Videos"):
    video_frame_dir = os.path.join(FRAME_ROOT_DIR, video_id)  # ✅ 각 영상의 프레임 폴더
    if not os.path.isdir(video_frame_dir):
        continue  # ✅ 폴더가 아닐 경우 건너뛰기

    frames = sorted(os.listdir(video_frame_dir))  # ✅ 해당 영상의 모든 프레임 가져오기
    metadata = []

    for frame_name in tqdm(frames, desc=f"Processing {video_id} Frames", leave=False):
        frame_path = os.path.join(video_frame_dir, frame_name)
        frame = cv2.imread(frame_path)

        # ✅ ArcFace 얼굴 인식
        faces = app.get(frame)

        # ✅ 타임스탬프 추출 (파일명에서 숫자 부분 가져오기)
        try:
            timestamp = int(frame_name.split("_")[-1].split(".")[0])
        except (IndexError, ValueError):
            print(f"⚠️ 잘못된 파일명 형식: {frame_name} → 기본값 0으로 설정")
            timestamp = 0  # 오류 발생 시 기본값 설정

        if not faces:
            metadata.append({
                "timestamp": timestamp,
                "video_id": video_id,
                "group": "UNKNOWN",
                "member": "Unknown",
                "confidence": 0.0,
                "bbox": None
            })
            continue

        # ✅ 얼굴 분석 및 메타데이터 저장
        for face in faces:
            bbox = face.bbox.astype(int)
            x1, y1, x2, y2 = bbox

            embedding = torch.tensor(face.embedding, dtype=torch.float32).unsqueeze(0).to(device)

            with torch.no_grad():
                group_logits, member_logits = model(embedding)

            # ✅ 확률값 변환
            group_probs = torch.softmax(group_logits, dim=1)
            member_probs = torch.softmax(member_logits, dim=1)

            predicted_group_id = torch.argmax(group_probs, dim=1).item()
            group_confidence = group_probs[0, predicted_group_id].item()

            if group_confidence < 0.7 or predicted_group_id >= len(GROUPS):
                predicted_group_name = "UNKNOWN"
                predicted_member_name = "Unknown"
                member_confidence = group_confidence
            else:
                predicted_group_name = GROUPS[predicted_group_id]
                predicted_member_id = torch.argmax(member_probs, dim=1).item()
                member_confidence = member_probs[0, predicted_member_id].item()

                if predicted_group_name in MEMBERS:
                    predicted_member_name = MEMBERS[predicted_group_name][predicted_member_id]
                else:
                    predicted_member_name = "Unknown"

            metadata.append({
                "timestamp": timestamp,
                "video_id": video_id,
                "group": predicted_group_name,
                "member": predicted_member_name,
                "confidence": member_confidence,
                "bbox": [x1, y1, x2, y2]  # ✅ bbox를 리스트로 변환하여 JSON 저장 가능하도록 함
            })

    # ✅ JSON 저장 전에 int64 값을 int로 변환하는 함수
    def convert_metadata_to_json(metadata):
        for entry in metadata:
            entry["timestamp"] = int(entry["timestamp"])  # ✅ timestamp 변환
            entry["confidence"] = float(entry["confidence"])  # ✅ float 변환

            if "bbox" in entry and entry["bbox"] is not None:
                entry["bbox"] = [int(coord) for coord in entry["bbox"]]  # ✅ bbox 변환

        return metadata  # ✅ 변환된 데이터 반환

    # ✅ 2️⃣ JSON 파일 저장 (영상별)
    save_path = os.path.join(SAVE_METADATA_DIR, f"{video_id}.json")
    metadata = convert_metadata_to_json(metadata)

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
        f.flush()
        os.fsync(f.fileno())

    print(f"✅ 저장 완료: {save_path} ({len(metadata)} entries)")

print("🎉 모든 영상의 얼굴 인식 데이터 저장 완료!")


In [ ]:
import os
import json

# 입력 및 출력 디렉토리 설정
input_dir = "/kaggle/input/metameat"
output_dir = "/kaggle/working/converted_metameat"
os.makedirs(output_dir, exist_ok=True)

# FPS 값 설정 (예: 29.97 FPS)
fps = 29.97

# 입력 디렉토리 내의 모든 JSON 파일 처리
for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        input_path = os.path.join(input_dir, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 각 항목의 timestamp를 초 단위로 변환 후 일반 반올림하여 정수로 변환
        for entry in data:
            if "timestamp" in entry:
                original_timestamp = entry["timestamp"]
                # 프레임 번호를 초로 변환하고 반올림한 후 정수로 변환
                new_timestamp = int(round(original_timestamp / fps))
                entry["timestamp"] = new_timestamp
                
        output_path = os.path.join(output_dir, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        
        print(f"Processed {filename} and saved to {output_path}")


=================================NJZ메타데이터만생성========================


In [2]:
pip install -q torch torchvision torchaudio numpy opencv-python onnxruntime insightface albumentations tqdm matplotlib pandas scikit-learn jsonlines sentence-transformers psycopg2-binary


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 9.1 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import torch
import numpy as np
import json
from insightface.app import FaceAnalysis
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
# --- 설정 ---
# 프레임이 저장된 폴더 (프로토타입 한 그룹 대상)
FRAME_DIR = "/kaggle/input/framenjz"
# 메타데이터 저장할 폴더
SAVE_METADATA_DIR = "/kaggle/working/metadata"
os.makedirs(SAVE_METADATA_DIR, exist_ok=True)

# FPS (예: 29.97 FPS)
fps = 29.97

# --- ArcFace 얼굴 인식 모델 초기화 ---
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
app.prepare(ctx_id=0 if torch.cuda.is_available() else -1)


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 84827.03KB/s] 
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [5]:
# --- 학습된 Hierarchical ArcFace 모델 로드 ---
MODEL_PATH = "/kaggle/input/realbest/2best_hierarchical_arcface(c).pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

class HierarchicalArcFaceModel(torch.nn.Module):
    def __init__(self, embedding_size=512, num_groups=6, num_members=[2, 2, 4, 4, 2, 1]):
        super(HierarchicalArcFaceModel, self).__init__()
        self.global_fc = torch.nn.Sequential(
            torch.nn.Linear(embedding_size, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
        )
        self.group_head = torch.nn.Linear(512, num_groups)  # 그룹 분류
        self.member_heads = torch.nn.ModuleList(
            [torch.nn.Linear(512, num_members[i]) for i in range(num_groups)]
        )

    def forward(self, x):
        features = self.global_fc(x)
        group_logits = self.group_head(features)  # 그룹 예측
        predicted_group = torch.argmax(group_logits, dim=1)  # 예측된 그룹

        batch_size = features.shape[0]
        member_logits_padded = torch.zeros(batch_size, max([h.out_features for h in self.member_heads])).to(features.device)
        for i in range(batch_size):
            group_idx = predicted_group[i].item()
            member_logits = self.member_heads[group_idx](features[i].unsqueeze(0))
            member_logits_padded[i, :member_logits.shape[1]] = member_logits
        return group_logits, member_logits_padded

model = HierarchicalArcFaceModel().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

<ipython-input-5-e8d2ec46cd35>:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=device))


HierarchicalArcFaceModel(
  (global_fc): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
  )
  (group_head): Linear(in_features=512, out_features=6, bias=True)
  (member_heads): ModuleList(
    (0-1): 2 x Linear(in_features=512, out_features=2, bias=True)
    (2-3): 2 x Linear(in_features=512, out_features=4, bias=True)
    (4): Linear(in_features=512, out_features=2, bias=True)
    (5): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [6]:
# 그룹 및 멤버 정보 (우리는 "NJZ" 그룹만 사용)
GROUPS = ["EXO", "IVE", "NJZ", "RED", "SK", "UNKNOWN"]
MEMBERS = {
    "EXO": ["D.O", "Kai"],
    "IVE": ["An Yujin", "Jang Wonyoung"],
    "NJZ": ["Danielle", "Gang Harin", "Kim Minji", "Pham Hanni"],
    "RED": ["Irene", "Joy", "Seulgi", "Wendy"],
    "SK": ["Hyunjin", "Felix"],
    "UNKNOWN": ["Unknown"]
}


In [7]:
# --- 메타데이터 생성 ---
metadata = []
frames = sorted(os.listdir(FRAME_DIR))

for frame_name in tqdm(frames, desc="Processing Frames"):
    # 파일명이 ".jpg"여야 함
    if not frame_name.lower().endswith(".jpg"):
        continue
    frame_path = os.path.join(FRAME_DIR, frame_name)
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    # 파일명 형식: FuJ1RiLoq-M_frame_XXXX.jpg
    try:
        # 예: "FuJ1RiLoq-M_frame_0029.jpg"에서 "0029" 추출
        frame_number = int(frame_name.split("_")[-1].split(".")[0])
    except Exception as e:
        print(f"Error parsing frame number for {frame_name}: {e}")
        continue

    # 프레임 번호를 FPS로 나누어 초 단위로 변환 (반올림)
    timestamp = int(round(frame_number / fps))

    # 얼굴 인식 수행
    faces = app.get(frame)
    if not faces:
        continue  # 얼굴이 인식되지 않으면 건너뜁니다.

    for face in faces:
        bbox = face.bbox.astype(int)
        x1, y1, x2, y2 = bbox

        # 얼굴 임베딩 추출 및 모델 예측
        embedding_tensor = torch.tensor(face.embedding, dtype=torch.float32).unsqueeze(0).to(device)
        with torch.no_grad():
            group_logits, member_logits = model(embedding_tensor)
        group_probs = torch.softmax(group_logits, dim=1)
        predicted_group_id = torch.argmax(group_probs, dim=1).item()
        predicted_group_name = GROUPS[predicted_group_id]

        # 오직 "NJZ" 그룹만 저장 (그렇지 않으면 continue)
        if predicted_group_name != "NJZ":
            continue

        member_probs = torch.softmax(member_logits, dim=1)
        predicted_member_id = torch.argmax(member_probs, dim=1).item()
        member_confidence = member_probs[0, predicted_member_id].item()

        predicted_member_name = MEMBERS[predicted_group_name][predicted_member_id]

        # 메타데이터 저장 (여기서 timestamp는 이미 정수 초 단위임)
        metadata.append({
            "timestamp": timestamp,
            "video_id": "FuJ1RiLoq-M",
            "group": predicted_group_name,
            "member": predicted_member_name,
            "confidence": float(torch.max(group_probs).item()),
            "bbox": [int(x1), int(y1), int(x2), int(y2)]
        })

        print(f"Processed {frame_name}: {predicted_group_name} - {predicted_member_name}")


Processing Frames:   0%|          | 0/1135 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
Processing Frames:   0%|          | 1/1135 [00:00<16:31,  1.14it/s]

Processed FuJ1RiLoq-M_frame_0000.jpg: NJZ - Gang Harin


Processing Frames:   0%|          | 2/1135 [00:01<15:42,  1.20it/s]

Processed FuJ1RiLoq-M_frame_0029.jpg: NJZ - Gang Harin


Processing Frames:   0%|          | 3/1135 [00:02<15:39,  1.20it/s]

...

Processed FuJ1RiLoq-M_frame_9918.jpg: NJZ - Danielle
Processed FuJ1RiLoq-M_frame_9918.jpg: NJZ - Pham Hanni


Processing Frames: 100%|█████████▉| 1134/1135 [13:41<00:01,  1.18s/it]

Processed FuJ1RiLoq-M_frame_9947.jpg: NJZ - Pham Hanni
Processed FuJ1RiLoq-M_frame_9947.jpg: NJZ - Gang Harin
Processed FuJ1RiLoq-M_frame_9947.jpg: NJZ - Danielle
Processed FuJ1RiLoq-M_frame_9947.jpg: NJZ - Danielle


Processing Frames: 100%|██████████| 1135/1135 [13:42<00:00,  1.38it/s]

Processed FuJ1RiLoq-M_frame_9976.jpg: NJZ - Kim Minji
Processed FuJ1RiLoq-M_frame_9976.jpg: NJZ - Pham Hanni


In [8]:
# --- 메타데이터 저장 ---
save_path = os.path.join(SAVE_METADATA_DIR, "face_metadata_NJZ.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

print(f"Metadata saved to {save_path}")

Metadata saved to /kaggle/working/metadata/face_metadata_NJZ.json


후처리

In [9]:
import json

# 입력 및 출력 파일 경로 설정
input_path = "/kaggle/working/metadata/face_metadata_NJZ.json"
output_path = "/kaggle/working/metadata/face_metadata_NJZ_processed.json"

# JSON 데이터 로드
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 1. confidence가 0.6 미만인 항목 제거
filtered_data = [record for record in data if record.get("confidence", 0) >= 0.6]

# 2. 같은 timestamp와 같은 member에 대해, confidence가 가장 높은 항목만 남기기
# 딕셔너리 키: (timestamp, member)
grouped = {}
for record in filtered_data:
    key = (record.get("timestamp"), record.get("member"))
    # 기존 값이 없거나, 현재 record의 confidence가 더 높으면 갱신
    if key not in grouped or record.get("confidence") > grouped[key].get("confidence"):
        grouped[key] = record

# 그룹화한 결과를 리스트로 변환
processed_data = list(grouped.values())

# 결과 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(processed_data, f, indent=4, ensure_ascii=False)

print(f"Processed metadata saved to {output_path}")


Processed metadata saved to /kaggle/working/metadata/face_metadata_NJZ_processed.json


meta(Arc) + meta(EILEV)

In [2]:
import json

# --- 파일 경로 설정 ---
# 영상 설명 세그먼트 메타데이터 파일 (예: 5초 단위 분할된 데이터)
segments_file = "/kaggle/input/meta-combine/video_captions_opt(beta).json"  
# 얼굴 인식 메타데이터 파일 (예: NJZ 그룹만 남은 상태)
faces_file = "/kaggle/input/meta-combine/face_metadata_NJZ_processed.json"  
# 병합된 결과를 저장할 파일
output_file = "/kaggle/working/metadata/combined_metadata.json"

# --- 데이터 로드 ---
with open(segments_file, "r", encoding="utf-8") as f:
    segments = json.load(f)

with open(faces_file, "r", encoding="utf-8") as f:
    faces = json.load(f)

# --- 중복 얼굴 필터 함수 ---
def filter_duplicate_faces(face_list):
    """
    동일 timestamp와 member를 가진 항목이 여러 개 있으면, confidence가 가장 높은 것만 남김.
    """
    face_dict = {}
    for face in face_list:
        key = (face.get("timestamp"), face.get("member"))
        if key not in face_dict or face.get("confidence", 0) > face_dict[key].get("confidence", 0):
            face_dict[key] = face
    return list(face_dict.values())

# --- 세그먼트별 얼굴 데이터 병합 ---
for seg in segments:
    # 영상 세그먼트는 예: { "segment": 0, "start_time": 0.0, "end_time": 5.0, "caption": "..." }
    start_time = seg.get("start_time", 0)
    end_time = seg.get("end_time", 0)
    
    # 해당 시간 구간 [start_time, end_time) 에 속하는 얼굴 데이터 선택
    seg_faces = [face for face in faces if start_time <= face.get("timestamp", 0) < end_time]
    
    # 동일 timestamp, member 중 confidence가 높은 항목만 남김
    seg_faces = filter_duplicate_faces(seg_faces)
    
    # 세그먼트에 얼굴 메타데이터 추가
    seg["faces"] = seg_faces




In [4]:
import os
import json

output_file = "/kaggle/working/metadata/combined_metadata.json"
os.makedirs(os.path.dirname(output_file), exist_ok=True)  # 디렉터리 생성

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(segments, f, indent=4, ensure_ascii=False)

print(f"Combined metadata saved to {output_file}")


# --- 병합된 결과 저장 ---
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(segments, f, indent=4, ensure_ascii=False)

print(f"Combined metadata saved to {output_file}")

Combined metadata saved to /kaggle/working/metadata/combined_metadata.json
Combined metadata saved to /kaggle/working/metadata/combined_metadata.json
